In [1]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from collections import defaultdict
import torch
from ultralytics import YOLO
from sklearn.preprocessing import StandardScaler

In [ ]:



### ========== إعدادات أساسية ==========

VIDEO_PATH = "BallVid/Untitled video - Made with Clipchamp (4).mp4"
MODEL_PATH = r"best (1).pt"


REFERENCE_IMAGE_PATH = "BallVid/تنزيل.jpeg"  

In [ ]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import os
import time

# ========== إعدادات أساسية ==========
# --- مسارات الملفات ---
MODEL_PATH = 'best (1).pt'  
VIDEO_PATH = 'BallVid/Untitled video - Made with Clipchamp (6).mp4' 
OUTPUT_VIDEO_PATH = "output_video_final_v3_improved.mp4" 

# --- إعدادات الكشف والتعرف ---
YOLO_CONF_THRESHOLD = 0.45  
PERSON_CONF_THRESHOLD = 0.50 
REFEREE_CONF_THRESHOLD = 0.65 # حد ثقة الحكم
PERSON_REFEREE_IOU_THRESHOLD = 0.3 # حد التقاطع لاستبعاد الشخص كلاعب إذا تداخل مع حكم مؤكد

# --- إعدادات فصل الفرق (KMeans) ---
KMEANS_WARMUP_SECONDS = 5 # زيادة طفيفة لوقت الإحماء لجمع ميزات أفضل
ROI_FOCUS_HEIGHT_PERCENT = 0.6 # نسبة ارتفاع اللاعب المستخدمة لاستخلاص اللون

# --- إعدادات تتبع اللاعبين (الذاكرة المؤقتة) ---
PLAYER_MEMORY_FRAMES = 20 # زيادة طفيفة لذاكرة اللاعب لربط أفضل عبر الإطارات المفقودة

# --- إعدادات تتبع الكرة ---
# استخدام ذاكرة أكبر قليلاً لتنعيم أفضل لمسار الكرة مع الحفاظ على الاستجابة
MAX_BALL_POSITIONS_MEMORY = 5
BALL_MAX_LOST_SECONDS = 0.6 # المدة بالثواني قبل اعتبار الكرة مفقودة تمامًا
BALL_VELOCITY_THRESHOLD = 60 # حد سرعة الكرة لمنع القفزات غير المنطقية (قد يحتاج لتعديل)

# --- إعدادات حساب الاستحواذ والتمرير ---
# تقليل المسافة المطلوبة للاستحواذ بشكل كبير لتكون أكثر دقة
POSSESSION_THRESHOLD_DISTANCE = 45
# زيادة مدة التأكيد قليلاً لتقليل التغييرات السريعة الخاطئة
POSSESSION_CONFIRM_DURATION_SECONDS = 0.8
# حد مسافة إضافي للتحقق من وجود لاعب منافس قريب (لتحديد الحالات المتنازع عليها)
POSSESSION_AMBIGUITY_DISTANCE_FACTOR = 1.3 #  إذا كان لاعب منافس ضمن (Threshold * Factor)

# --- إعدادات العرض البصري ---
BBOX_CORNER_LENGTH_FACTOR = 0.15 # طول زاوية البوكس كنسبة من البعد الأصغر
BBOX_THICKNESS = 2
BALL_RADIUS = 5
BALL_BORDER_RADIUS = 7

# ========== تهيئة الجهاز ونموذج YOLO ==========
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
try:
    model = YOLO(MODEL_PATH).to(device)
    print(f"YOLO model '{MODEL_PATH}' loaded successfully on {device}.")
    CLASS_NAMES = model.names
    # التأكد من وجود الفئات المطلوبة
    required_classes = ['ball', 'person', 'player', 'referee']
    print("Model Classes:", CLASS_NAMES)
    # يمكنك إضافة فحص هنا إذا كانت الفئات غير موجودة
except Exception as e:
    print(f"Error loading YOLO model: {e}")
    exit()

# ========== تهيئة الفيديو ==========
if not os.path.exists(VIDEO_PATH):
    print(f"Error: Video file not found at {VIDEO_PATH}")
    exit()
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    print(f"Error: Could not open video file {VIDEO_PATH}")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)
if fps <= 0:
    print("Warning: Could not get FPS from video. Defaulting to 30 FPS.")
    fps = 30

# حساب القيم المعتمدة على FPS
KMEANS_WARMUP_FRAMES = int(fps * KMEANS_WARMUP_SECONDS)
BALL_MAX_FRAMES_LOST = int(fps * BALL_MAX_LOST_SECONDS)
POSSESSION_CONFIRM_DURATION_FRAMES = int(fps * POSSESSION_CONFIRM_DURATION_SECONDS)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# استخدام أبعاد الفيديو الأصلية أو تغييرها حسب الحاجة
output_width = frame_width # أو 1280 مثلاً
output_height = frame_height # أو 720 مثلاً

# التأكد من أن أبعاد الإخراج معقولة
if output_width <= 0 or output_height <= 0:
    print(f"Warning: Invalid original video dimensions ({frame_width}x{frame_height}). Using 640x640 for output.")
    output_width, output_height = 640, 640

fourcc = cv2.VideoWriter_fourcc(*'mp4v') # أو استخدم 'XVID'
writer = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (output_width, output_height))
print(f"Video properties: Input {frame_width}x{frame_height} @ {fps:.2f} FPS -> Output {output_width}x{output_height}")
print(f"KMeans Warmup: {KMEANS_WARMUP_FRAMES} frames.")
print(f"Ball Lost Threshold: {BALL_MAX_FRAMES_LOST} frames.")
print(f"Possession Confirm Duration: {POSSESSION_CONFIRM_DURATION_FRAMES} frames.")
print(f"Possession Distance Threshold: {POSSESSION_THRESHOLD_DISTANCE} pixels.")

# ========== أدوات مساعدة ==========

def calculate_iou(boxA, boxB):
    """Calculates Intersection over Union (IoU) between two bounding boxes."""
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    unionArea = float(boxAArea + boxBArea - interArea)
    return interArea / unionArea if unionArea > 0 else 0

def draw_fancy_bbox(img, bbox, color, thickness, corner_factor):
    """Draws a bounding box with corners emphasized."""
    x1, y1, x2, y2 = map(int, bbox)
    w, h = x2 - x1, y2 - y1
    corner_size = int(min(w, h) * corner_factor)

    # Ensure corner size is not larger than half the side length
    corner_size = min(corner_size, w // 2, h // 2)
    if corner_size <= 0 : # Don't draw if box is too small
        cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness) # Fallback to simple rect
        return

    # Top-left corner
    cv2.line(img, (x1, y1), (x1 + corner_size, y1), color, thickness)
    cv2.line(img, (x1, y1), (x1, y1 + corner_size), color, thickness)
    # Top-right corner
    cv2.line(img, (x2, y1), (x2 - corner_size, y1), color, thickness)
    cv2.line(img, (x2, y1), (x2, y1 + corner_size), color, thickness)
    # Bottom-left corner
    cv2.line(img, (x1, y2), (x1 + corner_size, y2), color, thickness)
    cv2.line(img, (x1, y2), (x1, y2 - corner_size), color, thickness)
    # Bottom-right corner
    cv2.line(img, (x2, y2), (x2 - corner_size, y2), color, thickness)
    cv2.line(img, (x2, y2), (x2, y2 - corner_size), color, thickness)


# ========== إعداد لكشف الفرق (KMeans) ==========
scaler = StandardScaler()
kmeans = None
kmeans_trained = False
color_features_buffer = []
player_memory = {} # تخزين {center_tuple: {'team': id, 'last_seen_frame': frame_count}}

# ========== إعدادات تتبع الكرة وحساب الاستحواذ والتمريرات ==========
ball_positions_temporal = [] # قائمة بمواقع الكرة المكتشفة مؤخراً (للتمهيد)
tracked_ball_center = None   # موقع الكرة المتتبع / الممهد
ball_lost_counter = 0        # عداد الإطارات منذ آخر مرة شوهدت فيها الكرة

# --- متغيرات حالة الاستحواذ المحسنة ---
NO_POSSESSION = -1
TEAM_0 = 0
TEAM_1 = 1
AMBIGUOUS = -2 # حالة جديدة عندما يكون الوضع غير واضح

current_possession_state = NO_POSSESSION # يمكن أن يكون TEAM_0, TEAM_1, NO_POSSESSION, AMBIGUOUS
confirmed_possessor_team = NO_POSSESSION # الفريق المؤكد أنه مستحوذ (بعد تجاوز مدة التأكيد)
confirmed_possessor_player_center = None # مركز اللاعب المؤكد أنه مستحوذ
last_confirmed_possessor_player_center = None # لتتبع التمريرات
last_confirmed_possessor_team = NO_POSSESSION # لتتبع التمريرات

# متغيرات لتتبع التغيير المحتمل في الاستحواذ
potential_possessor_team = NO_POSSESSION
potential_possession_start_frame = -1

# عدادات الإحصائيات
possession_frames_team0 = 0
possession_frames_team1 = 0
total_processed_frames = 0
pass_count_team0 = 0
pass_count_team1 = 0

# ========== الدالة الرئيسية للمعالجة ==========
def process_video():
    # جعل المتغيرات العامة قابلة للتعديل داخل الدالة
    global kmeans, kmeans_trained, color_features_buffer, player_memory
    global tracked_ball_center, ball_lost_counter, ball_positions_temporal
    global current_possession_state, confirmed_possessor_team, confirmed_possessor_player_center
    global last_confirmed_possessor_player_center, last_confirmed_possessor_team
    global potential_possessor_team, potential_possession_start_frame
    global possession_frames_team0, possession_frames_team1, total_processed_frames
    global pass_count_team0, pass_count_team1

    frame_count = 0
    start_time = time.time()

    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video or error reading frame.")
            break

        frame_count += 1
        original_frame_for_drawing = frame.copy() # احتفظ بنسخة للرسم النهائي بالأبعاد الأصلية

        # تغيير حجم الإطار للمعالجة بواسطة YOLO (إذا لزم الأمر)
        # هنا نستخدم نفس أبعاد الإخراج، لكن يمكن استخدام أبعاد أخرى مثل 640x640 للمعالجة فقط
        try:
            process_frame = cv2.resize(frame, (output_width, output_height))
        except cv2.error as e:
            print(f"Frame {frame_count}: Error resizing frame: {e}. Skipping frame.")
            continue

        # --- 1. الكشف باستخدام YOLO ---
        try:
            results = model(process_frame, verbose=False, conf=YOLO_CONF_THRESHOLD)
            detections = results[0] # النتائج للإطار الأول في الدفعة (وهو الوحيد هنا)
        except Exception as e:
            print(f"Frame {frame_count}: Error during YOLO inference: {e}. Skipping frame.")
            continue

        # --- 2. استخلاص وفلترة الكشوفات الأولية ---
        all_persons_raw = []
        all_referees_raw = []
        detected_balls_raw = []

        if detections.boxes is not None:
            for i in range(len(detections.boxes)):
                box = detections.boxes[i]
                cls_id = int(box.cls[0])
                conf = float(box.conf[0])
                label = CLASS_NAMES.get(cls_id, f"ID_{cls_id}").lower()
                bbox_coords = tuple(map(int, box.xyxy[0].cpu().numpy()))

                # قص الإحداثيات ضمن حدود الإطار
                x1, y1, x2, y2 = bbox_coords
                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(output_width, x2), min(output_height, y2)
                if x1 >= x2 or y1 >= y2: continue # تجاهل الصناديق غير الصالحة
                valid_bbox = (x1, y1, x2, y2)

                if "ball" in label:
                    # قد يكون هناك كشوفات متعددة للكرة، اختر الأعلى ثقة
                    detected_balls_raw.append({'box': valid_bbox, 'conf': conf})
                elif "referee" in label and conf >= REFEREE_CONF_THRESHOLD:
                    all_referees_raw.append({'box': valid_bbox, 'conf': conf})
                elif ("person" in label or "player" in label) and conf >= PERSON_CONF_THRESHOLD:
                    all_persons_raw.append({'box': valid_bbox, 'conf': conf}) # تخزين الثقة قد يكون مفيدًا

        # --- 3. تحديد الكرة وتتبعها ---
        best_ball_box = None
        current_ball_center_raw = None
        ball_detected_this_frame = False
        if detected_balls_raw:
            ball_detected_this_frame = True
            detected_balls_raw.sort(key=lambda x: x['conf'], reverse=True) # ترتيب حسب الثقة
            best_ball_box = detected_balls_raw[0]['box']
            bx1, by1, bx2, by2 = best_ball_box
            current_ball_center_raw = ((bx1 + bx2) // 2, (by1 + by2) // 2)
            #print(f"Frame {frame_count}: Ball detected at {current_ball_center_raw}") # Debug

        # تحديث تتبع الكرة (التنعيم والتعامل مع الفقدان)
        if ball_detected_this_frame:
            ball_lost_counter = 0 # إعادة تعيين عداد الفقدان
            ball_positions_temporal.append(current_ball_center_raw)
            # الحفاظ على طول الذاكرة
            if len(ball_positions_temporal) > MAX_BALL_POSITIONS_MEMORY:
                ball_positions_temporal.pop(0)

            # حساب المركز الممهد
            if ball_positions_temporal:
                smoothed_center_np = np.mean(ball_positions_temporal, axis=0)
                new_tracked_center = tuple(smoothed_center_np.astype(int))

                # التحقق من السرعة لمنع القفزات الكبيرة
                if tracked_ball_center is not None:
                    movement = np.linalg.norm(np.array(new_tracked_center) - np.array(tracked_ball_center))
                    if movement < BALL_VELOCITY_THRESHOLD:
                        tracked_ball_center = new_tracked_center
                    else:
                        # السرعة عالية جداً، قد يكون خطأ كشف، احتفظ بالموقع القديم مؤقتًا
                        # وأزل الموقع الجديد الذي سبب القفزة من الذاكرة المؤقتة
                        if len(ball_positions_temporal) > 0 : ball_positions_temporal.pop(-1) # إزالة الأخير
                        # لا نحدث tracked_ball_center هنا، نستخدم القيمة السابقة
                        # print(f"Frame {frame_count}: Ball velocity threshold exceeded ({movement:.1f}). Ignoring new position.") # Debug
                else:
                    tracked_ball_center = new_tracked_center # أول كشف أو بعد فقدان طويل

            # تحديث الموقع السابق للتحقق من السرعة في الإطار التالي
            # previous_ball_position = tracked_ball_center # اسم المتغير القديم

        else: # لم يتم كشف الكرة في هذا الإطار
            ball_lost_counter += 1
            if ball_lost_counter > BALL_MAX_FRAMES_LOST:
                # الكرة مفقودة لفترة طويلة جداً
                tracked_ball_center = None
                ball_positions_temporal = [] # مسح الذاكرة
                # print(f"Frame {frame_count}: Ball lost.") # Debug
            # إذا لم تتجاوز الحد، tracked_ball_center يحتفظ بقيمته الأخيرة (تنبؤ بسيط)


        # --- 4. معالجة الحكام واللاعبين ---
        confirmed_referee_boxes = [r['box'] for r in all_referees_raw]
        players_for_clustering = [] # لتخزين اللاعبين المؤهلين للتصنيف

        # فلترة الأشخاص الذين يتداخلون بشكل كبير مع الحكام المؤكدين
        for p_det in all_persons_raw:
            p_box = p_det['box']
            is_likely_referee = False
            for r_box in confirmed_referee_boxes:
                if calculate_iou(p_box, r_box) > PERSON_REFEREE_IOU_THRESHOLD:
                    is_likely_referee = True
                    # print(f"Frame {frame_count}: Person {p_box} overlaps referee {r_box}. Filtering out.") # Debug
                    break
            if not is_likely_referee:
                players_for_clustering.append(p_det) # إضافة اللاعب المحتمل

        # --- 5. استخلاص ميزات اللون للاعبين ---
        player_features_this_frame = []
        player_info_for_kmeans = [] # تخزين {'box': bbox, 'center': (cx, cy), 'feature_index': idx}

        for player_det in players_for_clustering:
            x1, y1, x2, y2 = player_det['box']
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2
            center = (cx, cy)

            # تحديد منطقة الاهتمام (ROI) لاستخلاص اللون (الجزء العلوي من اللاعب)
            roi_y1 = y1
            roi_y2 = y1 + int((y2 - y1) * ROI_FOCUS_HEIGHT_PERCENT)
            roi_y2 = min(roi_y2, y2) # التأكد من عدم تجاوز الحد السفلي
            # التأكد من أن ROI لها أبعاد صالحة
            if roi_y1 < roi_y2 and x1 < x2:
                roi = process_frame[roi_y1:roi_y2, x1:x2]
                if roi.size > 0:
                    # حساب متوسط اللون في فضاء BGR
                    avg_color = np.mean(roi.reshape(-1, 3), axis=0)
                    player_features_this_frame.append(avg_color)
                    player_info_for_kmeans.append({
                        'box': player_det['box'],
                        'center': center,
                        'feature_index': len(player_features_this_frame) - 1
                    })

        # --- 6. تدريب KMeans وتصنيف الفرق ---
        team_assignments = {} # {feature_index: team_id (0 or 1)}

        if player_features_this_frame:
            if not kmeans_trained:
                # إضافة الميزات الجديدة للمخزن المؤقت
                color_features_buffer.extend(player_features_this_frame)
                # التحقق مما إذا كان وقت الإحماء قد انتهى والمخزن المؤقت يحتوي على بيانات كافية
                if frame_count >= KMEANS_WARMUP_FRAMES and len(color_features_buffer) >= 10: # ต้องการ على الأقل 10 نقاط لتدريب أولي جيد
                    cf_array = np.array(color_features_buffer)
                    if cf_array.shape[0] >= 2: # KMeans تحتاج على الأقل عينة واحدة لكل عنقود
                        try:
                            scaled_features = scaler.fit_transform(cf_array)
                            kmeans = KMeans(n_clusters=2, random_state=42, n_init=15, max_iter=300, tol=1e-4).fit(scaled_features)
                            kmeans_trained = True
                            print(f"--- KMeans trained successfully at frame {frame_count} with {len(color_features_buffer)} samples. ---")
                            # مسح المخزن المؤقت بعد التدريب الأول (اختياري، يمكن الاحتفاظ به لتحديث مستمر)
                            # color_features_buffer = []
                        except ValueError as e:
                            print(f"Frame {frame_count}: Error training KMeans (possibly insufficient samples): {e}. Resetting buffer.")
                            color_features_buffer = [] # مسح المخزن المؤقت إذا فشل التدريب
                    else:
                        # print(f"Frame {frame_count}: Not enough unique samples in buffer yet ({cf_array.shape[0]})") # Debug
                        pass

            if kmeans_trained: # إذا تم تدريب النموذج
                try:
                    pf_array = np.array(player_features_this_frame)
                    if pf_array.shape[0] > 0:
                        scaled_pf = scaler.transform(pf_array) # استخدام الـ scaler المدرب
                        team_labels = kmeans.predict(scaled_pf)
                        team_assignments = {info['feature_index']: label for info, label in zip(player_info_for_kmeans, team_labels)}
                        # print(f"Frame {frame_count}: Assigned teams: {team_assignments}") # Debug
                except Exception as e:
                    print(f"Frame {frame_count}: Error predicting teams: {e}")
                    # في حالة حدوث خطأ، لا يتم تعيين فرق لهذا الإطار
                    team_assignments = {}

        # --- 7. تحديث ذاكرة اللاعبين وتوزيعهم النهائي ---
        current_frame_player_centers = set()
        players_by_team = {TEAM_0: [], TEAM_1: []} # تخزين {'box': bbox, 'center': center} لكل فريق

        for info in player_info_for_kmeans:
            feature_idx = info['feature_index']
            center = info['center']
            box = info['box']
            current_frame_player_centers.add(center)
            assigned_team_id = -1

            # 1. هل تم تصنيفه بواسطة KMeans في هذا الإطار؟
            if feature_idx in team_assignments:
                assigned_team_id = team_assignments[feature_idx]
                # تحديث الذاكرة
                player_memory[center] = {'team': assigned_team_id, 'last_seen_frame': frame_count}
                # print(f"Frame {frame_count}: Player {center} classified as Team {assigned_team_id} by KMeans.") # Debug
            # 2. إذا لم يصنف، هل هو موجود في الذاكرة من إطارات قريبة؟
            elif center in player_memory:
                # البحث عن أقرب مركز في الذاكرة (قد تكون هناك حركة طفيفة)
                # هذا الجزء يمكن تحسينه بتتبع حقيقي، لكن الآن نستخدم تطابق المركز الدقيق أو القريب جدا
                if (frame_count - player_memory[center]['last_seen_frame']) <= PLAYER_MEMORY_FRAMES:
                    assigned_team_id = player_memory[center]['team']
                    # تحديث وقت الرؤية
                    player_memory[center]['last_seen_frame'] = frame_count
                    # print(f"Frame {frame_count}: Player {center} assigned Team {assigned_team_id} from memory.") # Debug
                # else: # إذا كان قديماً جداً في الذاكرة، تجاهله مؤقتًا
                    # print(f"Frame {frame_count}: Player {center} found in memory but too old ({frame_count - player_memory[center]['last_seen_frame']} frames).") # Debug

            # إضافة اللاعب إلى قائمة فريقه إذا تم تعيين فريق له
            if assigned_team_id in [TEAM_0, TEAM_1]:
                players_by_team[assigned_team_id].append({'box': box, 'center': center})

        # --- تنظيف الذاكرة من اللاعبين الذين لم تتم رؤيتهم لفترة طويلة ---
        centers_to_remove = [
            center for center, data in player_memory.items()
            if center not in current_frame_player_centers and (frame_count - data['last_seen_frame']) > PLAYER_MEMORY_FRAMES
        ]
        for center in centers_to_remove:
            # print(f"Frame {frame_count}: Removing player {center} from memory (last seen {player_memory[center]['last_seen_frame']}).") # Debug
            del player_memory[center]


        # --- 8. حساب الاستحواذ (منطق محسن) ---
        if not kmeans_trained:
             # لا يمكن حساب الاستحواذ قبل تدريب نموذج الفرق
             current_possession_state = NO_POSSESSION
             confirmed_possessor_team = NO_POSSESSION
             confirmed_possessor_player_center = None
             potential_possessor_team = NO_POSSESSION
             potential_possession_start_frame = -1
        elif tracked_ball_center is None:
            # الكرة غير مرئية / مفقودة
            current_possession_state = NO_POSSESSION
            # إذا فقدت الكرة، نفقد تأكيد الاستحواذ السابق ونوقف أي تغيير محتمل
            confirmed_possessor_team = NO_POSSESSION
            confirmed_possessor_player_center = None
            potential_possessor_team = NO_POSSESSION
            potential_possession_start_frame = -1
            # print(f"Frame {frame_count}: Ball not tracked, resetting possession.") # Debug
        else: # الكرة متتبعة و KMeans مدرب
            min_dist = float('inf')
            closest_player_info = None # {'center': center, 'team': team_id, 'dist': distance}
            opponent_nearby = False
            opponent_dist = float('inf')

            all_players_centers_teams = []
            for team_id, players in players_by_team.items():
                for p in players:
                    all_players_centers_teams.append({'center': p['center'], 'team': team_id})

            if not all_players_centers_teams:
                 # لا يوجد لاعبون مكتشفون/مصنفون في الإطار
                 current_possession_state = NO_POSSESSION # أو ربما نحتفظ بالحالة السابقة؟ NO_POSSESSION أوضح
                 confirmed_possessor_team = NO_POSSESSION
                 confirmed_possessor_player_center = None
                 potential_possessor_team = NO_POSSESSION
                 potential_possession_start_frame = -1
                 # print(f"Frame {frame_count}: No classified players detected.") # Debug
            else:
                # البحث عن أقرب لاعب
                for p_info in all_players_centers_teams:
                    dist = np.linalg.norm(np.array(tracked_ball_center) - np.array(p_info['center']))
                    if dist < min_dist:
                        min_dist = dist
                        closest_player_info = {'center': p_info['center'], 'team': p_info['team'], 'dist': dist}

                # التحقق مما إذا كان أقرب لاعب ضمن مسافة الاستحواذ
                if closest_player_info and min_dist < POSSESSION_THRESHOLD_DISTANCE:
                    candidate_team = closest_player_info['team']
                    candidate_center = closest_player_info['center']

                    # التحقق من وجود خصم قريب (حالة التنازع)
                    ambiguity_threshold = POSSESSION_THRESHOLD_DISTANCE * POSSESSION_AMBIGUITY_DISTANCE_FACTOR
                    for p_info in all_players_centers_teams:
                        if p_info['team'] != candidate_team:
                            dist_to_opponent = np.linalg.norm(np.array(tracked_ball_center) - np.array(p_info['center']))
                            if dist_to_opponent < ambiguity_threshold:
                                opponent_nearby = True
                                opponent_dist = min(opponent_dist, dist_to_opponent)
                                break # يكفي وجود خصم واحد قريب

                    if opponent_nearby:
                        # حالة تنازع: خصم قريب جداً
                        current_possession_state = AMBIGUOUS
                        # لا نؤكد الاستحواذ لأي فريق ونوقف مؤقت التغيير المحتمل
                        potential_possessor_team = NO_POSSESSION
                        potential_possession_start_frame = -1
                        confirmed_possessor_team = NO_POSSESSION # فقدان التأكيد
                        confirmed_possessor_player_center = None
                        # print(f"Frame {frame_count}: Ambiguous possession. Closest T{candidate_team} ({min_dist:.1f}px), Opponent nearby ({opponent_dist:.1f}px < {ambiguity_threshold:.1f}px).") # Debug
                    else:
                        # لاعب واحد قريب بوضوح (لا يوجد خصم قريب جداً)
                        # الآن نطبق منطق تأكيد الاستحواذ الزمني
                        if candidate_team == confirmed_possessor_team:
                            # نفس الفريق لا يزال هو الأقرب، استمرار الاستحواذ المؤكد
                            current_possession_state = candidate_team
                            confirmed_possessor_player_center = candidate_center # تحديث اللاعب المستحوذ
                            # إعادة تعيين أي تغيير محتمل كان قيد الانتظار
                            potential_possessor_team = NO_POSSESSION
                            potential_possession_start_frame = -1
                            # print(f"Frame {frame_count}: Possession confirmed continuation for Team {candidate_team} by player {candidate_center}.") # Debug
                        else:
                            # فريق مختلف هو الأقرب، هل بدأنا تتبع هذا التغيير المحتمل؟
                            if potential_possessor_team == candidate_team:
                                # نعم، نفس الفريق المنافس لا يزال هو الأقرب
                                if potential_possession_start_frame != -1 and \
                                   (frame_count - potential_possession_start_frame) >= POSSESSION_CONFIRM_DURATION_FRAMES:
                                    # المدة اكتملت، تأكيد تغيير الاستحواذ
                                    # print(f"--- Possession CHANGE confirmed to Team {candidate_team} at frame {frame_count} ---") # Debug
                                    current_possession_state = candidate_team
                                    confirmed_possessor_team = candidate_team
                                    confirmed_possessor_player_center = candidate_center
                                    # إعادة تعيين مؤقت التغيير
                                    potential_possessor_team = NO_POSSESSION
                                    potential_possession_start_frame = -1
                                else:
                                    # المدة لم تكتمل بعد، لا يزال الاستحواذ مع الفريق السابق مؤقتًا
                                    current_possession_state = confirmed_possessor_team # احتفظ بالحالة المؤكدة السابقة
                                    # لا نغير confirmed_possessor_team أو confirmed_possessor_player_center
                                    # print(f"Frame {frame_count}: Potential change to Team {candidate_team} ongoing ({frame_count - potential_possession_start_frame}/{POSSESSION_CONFIRM_DURATION_FRAMES} frames). Current state: {current_possession_state}") # Debug
                            else:
                                # فريق منافس جديد هو الأقرب، ابدأ مؤقت التغيير المحتمل
                                # print(f"Frame {frame_count}: Potential possession change detected. Candidate: Team {candidate_team}. Starting timer.") # Debug
                                potential_possessor_team = candidate_team
                                potential_possession_start_frame = frame_count
                                # الاستحواذ الحالي لا يزال مع الفريق المؤكد الأخير حتى يكتمل المؤقت
                                current_possession_state = confirmed_possessor_team
                                # لا نغير confirmed_possessor_team أو confirmed_possessor_player_center

                else: # لا يوجد لاعب قريب بما فيه الكفاية
                    current_possession_state = NO_POSSESSION
                    # فقدان تأكيد الاستحواذ وإيقاف أي تغيير محتمل
                    confirmed_possessor_team = NO_POSSESSION
                    confirmed_possessor_player_center = None
                    potential_possessor_team = NO_POSSESSION
                    potential_possession_start_frame = -1
                    # print(f"Frame {frame_count}: No player within possession threshold ({min_dist:.1f}px > {POSSESSION_THRESHOLD_DISTANCE}px).") # Debug


        # --- 9. حساب التمريرات (يعتمد على تغيير اللاعب المؤكد داخل نفس الفريق) ---
        if confirmed_possessor_player_center is not None and confirmed_possessor_team != NO_POSSESSION:
             # هل كان هناك لاعب مستحوذ مؤكد سابق من نفس الفريق؟
             if last_confirmed_possessor_player_center is not None and last_confirmed_possessor_team == confirmed_possessor_team:
                 # هل اللاعب الحالي مختلف عن اللاعب السابق؟ (مع تجنب عد تمريرة لنفس اللاعب إذا تحرك قليلاً)
                 # يمكن إضافة فحص مسافة هنا إذا كانت المراكز متقاربة جداً
                 if confirmed_possessor_player_center != last_confirmed_possessor_player_center:
                      # نعم، تمت تمريرة داخل نفس الفريق!
                      if confirmed_possessor_team == TEAM_0:
                          pass_count_team0 += 1
                      elif confirmed_possessor_team == TEAM_1:
                          pass_count_team1 += 1
                      # print(f"--- Pass Detected: Team {confirmed_possessor_team}, Frame {frame_count} (From {last_confirmed_possessor_player_center} to {confirmed_possessor_player_center}) ---") # Debug
                      # تحديث اللاعب الأخير *فقط* بعد تسجيل التمريرة
                      last_confirmed_possessor_player_center = confirmed_possessor_player_center
             else:
                  # هذا هو أول لاعب مؤكد لهذا الاستحواذ، أو جاء بعد فريق آخر أو عدم استحواذ
                  last_confirmed_possessor_player_center = confirmed_possessor_player_center
                  last_confirmed_possessor_team = confirmed_possessor_team
        # إذا لم يكن هناك لاعب مؤكد في هذا الإطار (بسبب فقدان الكرة، التنازع، أو لا لاعب قريب)
        # نقوم بإعادة تعيين اللاعب المؤكد الأخير لمنع حساب تمريرة خاطئة عند استعادة الاستحواذ
        elif confirmed_possessor_player_center is None:
             last_confirmed_possessor_player_center = None
             last_confirmed_possessor_team = NO_POSSESSION


        # --- 10. تحديث إحصائيات وقت الاستحواذ ---
        if kmeans_trained: # ابدأ العد فقط بعد تدريب Kmeans
            total_processed_frames += 1
            # نزيد العداد للفريق المؤكد فقط
            if confirmed_possessor_team == TEAM_0:
                possession_frames_team0 += 1
            elif confirmed_possessor_team == TEAM_1:
                possession_frames_team1 += 1

        # --- 11. الرسم على إطار الإخراج (استخدام النسخة الأصلية بالأبعاد الصحيحة) ---
        # تغيير حجم إطار العرض ليتناسب مع أبعاد الإخراج المطلوبة
        display_frame = cv2.resize(original_frame_for_drawing, (output_width, output_height))

        # ألوان الفرق (يمكن تعديلها)
        team_colors = {TEAM_0: (0, 200, 0), TEAM_1: (255, 100, 0)} # أخضر، برتقالي/أزرق
        referee_color = (0, 220, 220) # سماوي
        ball_color_default = (220, 220, 220) # رمادي فاتح
        ball_color_possession = {TEAM_0: team_colors[TEAM_0], TEAM_1: team_colors[TEAM_1]}
        ambiguous_color = (0, 255, 255) # أصفر

        # رسم الحكام
        for r_box in confirmed_referee_boxes:
            draw_fancy_bbox(display_frame, r_box, referee_color, BBOX_THICKNESS, BBOX_CORNER_LENGTH_FACTOR)
            # cv2.rectangle(display_frame, (r_box[0], r_box[1]), (r_box[2], r_box[3]), referee_color, 2)
            # cv2.putText(display_frame, "Referee", (r_box[0], r_box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, referee_color, 1)

        # رسم اللاعبين (باستخدام الصناديق الاحترافية)
        for team_id, players in players_by_team.items():
            color = team_colors.get(team_id)
            for p_info in players:
                draw_fancy_bbox(display_frame, p_info['box'], color, BBOX_THICKNESS, BBOX_CORNER_LENGTH_FACTOR)
                # رسم دائرة صغيرة في مركز اللاعب (اختياري للتصحيح)
                # cv2.circle(display_frame, p_info['center'], 3, color, -1)


        # رسم الكرة (مع تلوين حسب الاستحواذ)
        if tracked_ball_center:
            ball_render_center = tuple(map(int, tracked_ball_center))
            render_color = ball_color_default

            # تحديد لون الكرة بناءً على حالة الاستحواذ الحالية
            if current_possession_state == TEAM_0:
                render_color = ball_color_possession[TEAM_0]
            elif current_possession_state == TEAM_1:
                render_color = ball_color_possession[TEAM_1]
            elif current_possession_state == AMBIGUOUS:
                render_color = ambiguous_color # لون خاص للتنازع

            cv2.circle(display_frame, ball_render_center, BALL_RADIUS, render_color, -1)
            cv2.circle(display_frame, ball_render_center, BALL_BORDER_RADIUS, (255, 255, 255), 1) # حد أبيض رقيق

            # إضافة مؤشر تغيير محتمل (دائرة صفراء حول الكرة)
            if potential_possessor_team != NO_POSSESSION:
                 cv2.circle(display_frame, ball_render_center, BALL_BORDER_RADIUS + 3, ambiguous_color, 1)


        # --- 12. عرض معلومات الاستحواذ والتمريرات ---
        if kmeans_trained: # عرض الإحصائيات فقط بعد تدريب Kmeans
            bar_height = 35
            bar_y = 5 # تقليل المسافة من الأعلى
            stats_y = bar_y + bar_height + 15 # تقليل المسافة بين الشريط والنصوص

            # --- تطبيع النسب المئوية للاستحواذ ---
            total_possession_counted = possession_frames_team0 + possession_frames_team1
            if total_possession_counted > 0:
                display_percent_team0 = (possession_frames_team0 / total_possession_counted) * 100
                display_percent_team1 = (possession_frames_team1 / total_possession_counted) * 100
            else:
                display_percent_team0 = 0
                display_percent_team1 = 0

            # حساب عرض الأشرطة
            # نستخدم نصف العرض الكلي كأساس 100% لكل فريق
            team0_bar_width = int((output_width / 2) * (display_percent_team0 / 100))
            team1_bar_width = int((output_width / 2) * (display_percent_team1 / 100))

            # رسم خلفيات الأشرطة (ألوان باهتة)
            cv2.rectangle(display_frame, (0, bar_y), (output_width // 2, bar_y + bar_height), (0, 60, 0), -1) # أخضر داكن
            cv2.rectangle(display_frame, (output_width // 2, bar_y), (output_width, bar_y + bar_height), (60, 30, 0), -1) # أزرق/برتقالي داكن

            # رسم الأشرطة الفعلية
            cv2.rectangle(display_frame, (0, bar_y), (team0_bar_width, bar_y + bar_height), team_colors[TEAM_0], -1)
            cv2.rectangle(display_frame, (output_width - team1_bar_width, bar_y), (output_width, bar_y + bar_height), team_colors[TEAM_1], -1)

            # كتابة النسب المئوية
            text_team0 = f"Team 0: {display_percent_team0:.1f}%"
            text_team1 = f"Team 1: {display_percent_team1:.1f}%"
            # إضافة خلفية للنص لتحسين القراءة
            (tw0, th0), _ = cv2.getTextSize(text_team0, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
            (tw1, th1), _ = cv2.getTextSize(text_team1, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
            cv2.rectangle(display_frame, (5, bar_y + bar_height - th0 - 8), (5 + tw0 + 4, bar_y + bar_height-5), (0,0,0, 0.5), -1) # خلفية شفافة قليلاً
            cv2.rectangle(display_frame, (output_width - tw1 - 10, bar_y + bar_height - th1 - 8), (output_width - 5, bar_y + bar_height - 5), (0,0,0, 0.5), -1)

            cv2.putText(display_frame, text_team0, (8, bar_y + bar_height - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            cv2.putText(display_frame, text_team1, (output_width - tw1 - 8, bar_y + bar_height - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            # الخط الفاصل في المنتصف
            cv2.line(display_frame, (output_width // 2, bar_y), (output_width // 2, bar_y + bar_height), (255, 255, 255), 1)

            # كتابة عدد التمريرات
            pass_text_team0 = f"Passes: {pass_count_team0}"
            pass_text_team1 = f"Passes: {pass_count_team1}"
            cv2.putText(display_frame, pass_text_team0, (10, stats_y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            (text_width_p1, _), _ = cv2.getTextSize(pass_text_team1, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            cv2.putText(display_frame, pass_text_team1, (output_width - text_width_p1 - 10, stats_y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            # عرض حالة الاستحواذ الحالية أو التغيير المحتمل في المنتصف
            status_text = ""
            status_color = (255, 255, 255)
            if current_possession_state == AMBIGUOUS:
                 status_text = "Contested"
                 status_color = ambiguous_color
            elif potential_possessor_team != NO_POSSESSION and potential_possession_start_frame != -1:
                 frames_passed = frame_count - potential_possession_start_frame
                 time_passed_sec = frames_passed / fps
                 status_text = f"Team {potential_possessor_team} Close ({time_passed_sec:.1f}s / {POSSESSION_CONFIRM_DURATION_SECONDS:.1f}s)"
                 status_color = ambiguous_color # أصفر للإشارة للتحذير
            elif confirmed_possessor_team != NO_POSSESSION:
                 status_text = f"Team {confirmed_possessor_team} Possession"
                 status_color = team_colors[confirmed_possessor_team]
            # else: # No Possession
            #    status_text = "No Possession"

            if status_text:
                (text_w, text_h), _ = cv2.getTextSize(status_text, cv2.FONT_HERSHEY_SIMPLEX, 0.45, 1)
                text_x = output_width // 2 - text_w // 2
                text_y = stats_y # وضعها أسفل شريط النسبة المئوية
                 # إضافة خلفية بسيطة
                cv2.rectangle(display_frame, (text_x - 3 , text_y - text_h - 1), (text_x + text_w + 3, text_y + 3), (50, 50, 50), -1)
                cv2.putText(display_frame, status_text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, status_color, 1)

        # --- 13. كتابة الإطار الناتج ---
        try:
            writer.write(display_frame)
        except Exception as e:
            print(f"Error writing frame {frame_count} to video file: {e}")
            # قد تحتاج لإيقاف المعالجة هنا إذا تكررت الأخطاء

        # --- عرض الإطار (اختياري للتصحيح المباشر) ---
        # cv2.imshow("Processed Frame", display_frame)
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #    break

        # طباعة التقدم كل فترة
        if frame_count % int(fps * 10) == 0: # كل 10 ثواني تقريبًا
             elapsed_time = time.time() - start_time
             avg_fps = frame_count / elapsed_time
             print(f"Processed frame {frame_count} / Estimated Time: {int(elapsed_time // 60)}m {int(elapsed_time % 60)}s / Avg FPS: {avg_fps:.2f}")


    # --- الانتهاء ---
    end_time = time.time()
    total_time = end_time - start_time
    print("\n--------- Video Processing Finished ---------")
    print(f"Total processing time: {total_time:.2f} seconds for {frame_count} frames.")
    if frame_count > 0: print(f"Average processing FPS: {frame_count / total_time:.2f}")
    print(f"Output video saved to: {OUTPUT_VIDEO_PATH}")
    # طباعة الإحصائيات النهائية
    print("\n--- Final Statistics ---")
    if total_possession_counted > 0:
        final_perc_t0 = (possession_frames_team0 / total_possession_counted) * 100
        final_perc_t1 = (possession_frames_team1 / total_possession_counted) * 100
        print(f"Possession: Team 0: {final_perc_t0:.1f}% ({possession_frames_team0} frames) | Team 1: {final_perc_t1:.1f}% ({possession_frames_team1} frames)")
    else:
        print("Possession: No possession recorded (KMeans might not have trained or no players detected).")
    print(f"Passes: Team 0: {pass_count_team0} | Team 1: {pass_count_team1}")
    print("-------------------------------------------")

    cap.release()
    writer.release()
    cv2.destroyAllWindows()

# ========== نقطة بداية التنفيذ ==========
if __name__ == "__main__":
    process_video()

Using device: cuda
YOLO model 'best (1).pt' loaded successfully on cuda.
Model Classes: {0: 'Ball', 1: 'Person', 2: 'Referee'}
Video properties: Input 1920x1080 @ 30.00 FPS -> Output 1920x1080
KMeans Warmup: 150 frames.
Ball Lost Threshold: 18 frames.
Possession Confirm Duration: 24 frames.
Possession Distance Threshold: 45 pixels.
--- KMeans trained successfully at frame 150 with 2683 samples. ---
Processed frame 300 / Estimated Time: 0m 15s / Avg FPS: 18.91
Processed frame 600 / Estimated Time: 0m 27s / Avg FPS: 22.07
Processed frame 900 / Estimated Time: 0m 38s / Avg FPS: 23.42
Processed frame 1200 / Estimated Time: 0m 48s / Avg FPS: 24.50
End of video or error reading frame.

--------- Video Processing Finished ---------
Total processing time: 51.27 seconds for 1263 frames.
Average processing FPS: 24.63
Output video saved to: output_video_final_v3_improved.mp4

--- Final Statistics ---
Possession: Team 0: 100.0% (8 frames) | Team 1: 0.0% (0 frames)
Passes: Team 0: 6 | Team 1: 0
---

True
1
NVIDIA GeForce RTX 4060
